# Phase Circuit Optimization

Changing working directory from `./notebooks/` to `./`, in order to import the Python packages defined in the repository.

In [ ]:
import os
DEBUG = True
try:
    print("Original working directory: %s"%str(original_wd)) # type: ignore
    """
        You only get here if you---by mistake or otherwise---are re-running this cell, 
        in which case the working should not be changed again.
    """
except NameError:
    original_wd = os.getcwd()
    os.chdir('../')
print("Current working directory: %s"%str(os.getcwd()))

General purpose imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# import qiskit

Some utility code to display HTML elements and images/figures side-by-side:

In [ ]:
from io import BytesIO
from typing import Union
from IPython.display import Image, HTML # type: ignore
def figure_to_image(fig: plt.Figure, fmt: str = "png") -> Image:
    """ Converts a Matplotlib figure to a PNG IPython Image. """
    buffer = BytesIO()
    fig.savefig(buffer, format=fmt)
    buffer.seek(0)
    image_png = buffer.getvalue()
    buffer.close()
    return Image(image_png, format=fmt, embed=True)
def side_by_side(*elements: Union[str, Image]) -> HTML:
    """
        Returns an HTML Div element with the given elements
        displayed side by side. Accepts raw HTML code or
        IPython Image objects.
    """
    html = f"<div style='display:flex; align-items: center;'>"
    for el in elements:
        if isinstance(el, str):
            html += el
        elif isinstance(el, Image):
            html += f"<img src='data:image/png;base64,{el._repr_png_()}'/>"
    html += f"</div>"
    return HTML(html)

## Phase Circuit Optimization

The optimisation of a circuit $\mathcal{C}$ of mixed ZX phase gadgets proceeds through conjugation of the circuit by a suitably chosen block $U$ of CX gates, obtaining another circuit $U \circ \mathcal{C}' \circ U^\dagger$ (hopefully simpler overall).

This is done by a `PhaseCircuitOptimizer`, which can be instantiated by specifying:

- the original phase circuit to be optimized;
- a topology, constraining the CX circuit used for the optimization;
- a number of layers to use for the CX circuit.

In [ ]:
from pauliopt.phase import PhaseCircuitOptimizer

For our running example, we use a 3x3 grid qubit topology:

In [ ]:
from pauliopt.topologies import Topology
topology = Topology.grid(3, 3)
topology.draw(figsize=(3, 3))

We construct a small random phase circuit on the same qubits:

In [ ]:
from pauliopt.phase import PhaseCircuit, CXCircuit
orig_circuit = PhaseCircuit.random(topology.qubits, 6, rng_seed=0)
orig_circuit

To optimize it, we instantiate an optimizer with a single-layer CX circuit. The optional parameter `rng_seed` can be used to pass a seed to the RNG.

In [ ]:
num_cx_layers = 1
opt = PhaseCircuitOptimizer(orig_circuit, topology, num_cx_layers, rng_seed=0)

The topology, its qubits and the phase gadgets in the original circuit are made available through suitably named properties of the optimizer. The original circuit is not saved directly (because it is mutable).

In [ ]:
print(f"{opt.qubits = }", end="\n\n")
print(f"{opt.topology = }", end="\n\n")
print(f"{opt.original_gadgets = }", end="\n\n")

## Phase and CX Blocks

Circuit optimization is progressive. The currently optimized circuit is in the form $U \circ \mathcal{C}' \circ U^\dagger$, where $\mathcal{C}'$ is the **phase block** and $U$ is the **CX block**. The optimizer makes both available through suitably named properties (the phase block as a read-only view on a `PhaseCircuit`, the CX block as a read-only view on a `CXCircuit`).

At the beginning, the phase block is equal to the original circuit.

In [ ]:
opt.phase_block.to_svg(scale=0.8)

At the beginning, the CX block is empty (given number of layers, no CX gates).

In [ ]:
opt.cx_block.draw(figsize=(3, 3))

## Random CX Flips

Optimization is performed by simulated annealing. Random CX gates are flipped in the CX circuit $U$ for a given number of iterations, in an attempt to reduce the complexity of the overall optimized circuit $U \circ \mathcal{C}' \circ U$. The method `PhaseCircuitOptimizer.random_flip_cx()` is used to perform such a random flip: it modifies the internal phase circuit and CX circuit in place, returning information about the layer index and CX gate that was just flipped in case it has to be undone.

The method `PhaseCircuitOptimizer.flip_cx(layer_idx, ctrl, trgt)` is used to perform a specific CX gate flip, while the method `PhaseCircuitOptimizer.is_cx_flippable(layer_idx, ctrl, trgt)` can be used to check if a specific flip can be performed.

Here we see an example with a single random flip:

In [ ]:
opt = PhaseCircuitOptimizer(orig_circuit, topology, 1, rng_seed=0)
print("=== Phase block and CX block before random flip ===")
display(opt.phase_block.to_svg(scale=0.6))
opt.cx_block.draw(figsize=(3, 3))
layer_idx, gate = opt.random_flip_cx()
print(f"\n\n=== Phase block and CX block after flipping gate {gate} in layer {layer_idx} ===")
display(opt.phase_block.to_svg(scale=0.6))
opt.cx_block.draw(figsize=(3, 3))

Here we see another example, at the beginning and after 8 random flips:

In [ ]:
opt = PhaseCircuitOptimizer(orig_circuit, topology, 1, rng_seed=0)
print("=== Phase block and CX block before random flip ===")
display(opt.phase_block.to_svg(scale=0.6))
opt.cx_block.draw(figsize=(3, 3))
for _ in range(3):
    opt.random_flip_cx()
print(f"\n\n=== Phase block and CX block after 3 random flips ===")
display(opt.phase_block.to_svg(scale=0.6))
opt.cx_block.draw(figsize=(3, 3))

## Annealing - Prelude

Randomly flipping CX gates and hoping for the best is not much good as an optimisation strategy: simulated annealing is used instead as a global optimisation technique.

The method `PhaseCircuitOptimizeranneal(num_iters, temp_schedule, cost_fun)` allows a given number of iterations of simulated annealing to be performed on the circuit, for a given temperature schedule and cost function.
The method also accepts an optional keyword argument `loggers`, for detailed logging. 

## Temperature Schedule

A **temperature schedule** is any function which fits the `TempSchedule` protocol below: given the index `it` of the current iteration an the total number `num_iters` of iterations, it returns the temperature (as an `int` or `float` number).

In [ ]:
from typing import Protocol, Union
Number = Union[int, float]
class TempSchedule(Protocol):
    """
        Protocol for a temperature schedule.
        The temperature is a `number` computed from the iteration number `it`
        (starting from 0) and the total number of iterations `num_iter`.
    """

    def __call__(self, it: int, num_iters: int) -> Number:
        ...

The library offers 4 utility functions `temp_schedule_maker(t_init, t_final)` that produce different standard temperature schedules (linear, geometric, reciprocal and logarithmic) given initial and final temperatures. The schedules are taken from [this paper](https://link.springer.com/article/10.1007/BF00143921).

In [ ]:
from pauliopt.utils import (straight_temp_schedule, geometric_temp_schedule, 
                            reciprocal_temp_schedule, log_temp_schedule)

Here is a plot of the four temperature schedules over 100 iterations, for initial temperature 10 and final temperature 1. 

In [ ]:
num_iters = 100
plt.figure(figsize=(10,5))
for temp_schedule_maker in [straight_temp_schedule, geometric_temp_schedule, 
                            reciprocal_temp_schedule, log_temp_schedule]:
    temp_schedule = temp_schedule_maker(t_init=10, t_final=1)
    temp = [temp_schedule(it, num_iters) for it in range(num_iters)]
    plt.plot(range(num_iters), temp, label=temp_schedule_maker.__name__)
plt.legend()
plt.show()

## Cost Function

A **cost function** is any function which fits the `CostFun` protocol below: given a phase block and a CX block, it returns a cost in the form of an `int` or `float` number.

In [ ]:
from pauliopt.phase import PhaseCircuitView, CXCircuitView
class CostFun(Protocol):
    """
        Protocol for a cost function.
        The cost is a `float` computed from the phase block (readonly view)
        and CX block (readonly view) exposed by an instance of `PhaseCircuitOptimizer`.
    """

    def __call__(self, phase_block: PhaseCircuitView, cx_block: CXCircuitView) -> Number:
        ...

The library offers a utility function `cx_count_cost_fun(topology, circuit_rep=1)` that constructs such a cost function based on a topology and the number of times that the given circuit is going to be repeated (e.g. when the circuit is a single layer of a multi-layer ansatz).
The cost function counts the number of CX gates necessary to implement the optimized circuit $U \circ (\mathcal{C}')^{circuit\_rep} \circ U^\dagger$, where phase gadgets are implemented by a minimum spanning tree (MST) technique and long-range CX gates are implemented by a double-ladder of nearest neighbour CX gates ($2d-1$ NN CX gates where $d$ is the distance on the topology between control and target).

The library also offers a utility function `cx_count(gadget, topology)` that produces detailed information about the CX count for a single phase gadget on a given topology using this MST technique.

In [ ]:
from pauliopt.phase import cx_count_cost_fun, cx_count
# In our examples, we consider a single repetition of the circuit:
cost_fun = cx_count_cost_fun(topology, circuit_rep=1)

To understand how the cost of a single phase gadget is computed, we consider the following example on a 3x3 grid topology.

In [ ]:
test_circ = PhaseCircuit.random(topology.qubits, 1, rng_seed=0)
test_circ

The phase gadget above spans qubits `{0, 1, 5, 7, 8}`. One possible minimal spanning tree implementation involves two mirror ladders of the following CX gates (with a phase gate on qubit 7 in between):

- 1 CX gate between 0 and 1 (dist: 1)
- 3 CX gate between 1 and 5 (dist: 2)
- 1 CX gate between 5 and 8 (dist: 1)
- 1 CX gate between 8 and 7 (dist: 1)

Each ladder has 6 CX gates, so the total CX count for the phase gadget is 12.

In [ ]:
test_gadget = test_circ.gadgets[0]
cx_counts, mst = cx_count(test_gadget, topology)
print(f"Overall CX count for phase gadget: {sum(cx_counts)}")
print(f"MST for phase gadget: {mst}")
print(f"CX counts for MST edges (both ladders): {cx_counts}")

Let's now consider a second example:

In [ ]:
test_circ = PhaseCircuit.random(topology.qubits, 1, rng_seed=1)
test_circ

The phase gadget above spans qubits `{0, 1, 2, 4, 5, 6, 8}`. One possible minimal spanning tree implementation involves two mirror ladders of the following CX gates (with a phase gate on qubit 7 in between):

- 1 CX gate between 0 and 1 (dist: 1)
- 1 CX gate between 1 and 2 (dist: 1)
- 1 CX gate between 1 and 4 (dist: 1)
- 1 CX gate between 2 and 5 (dist: 1)
- 1 CX gate between 5 and 8 (dist: 1)
- 3 CX gates between 0 and 6 (dist: 2)

Each ladder has 8 CX gates, so the total CX count for the phase gadget is 16.

In [ ]:
test_gadget = test_circ.gadgets[0]
cx_counts, mst = cx_count(test_gadget, topology)
print(f"Overall CX count for phase gadget: {sum(cx_counts)}")
print(f"MST for phase gadget: {mst}")
print(f"CX counts for MST edges (both ladders): {cx_counts}")

Now let's look at an example of cost reduction in a circuit with several phase gadgets.
Consider the example from before:

In [ ]:
opt = PhaseCircuitOptimizer(orig_circuit, topology, 1, rng_seed=1)
cost = cost_fun(opt.phase_block, opt.cx_block)
print(f"Cost before flip: {cost}")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.phase_block.gadgets]}")
print(f"  - CX blocks CX count: {2*opt.cx_block.num_gates}")
display(opt.phase_block.to_svg(scale=0.4))
opt.cx_block.draw(figsize=(2, 2))

If we conjugate the circuit by a CX gate with control 0 and target 1, the cost changes as follows:

- we remove 4 CX gates from the phase block (3rd and 5th phase gadgets lose a leg);
- we add 2 CX gates to the CX blocks;

Overall, the cost is reduced by 2. It would be reduced by `4*circuit_rep-2` in the general case.

In [ ]:
opt.flip_cx(0, 1, 0)
cost = cost_fun(opt.phase_block, opt.cx_block)
print(f"Cost after flip: {cost}")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.phase_block.gadgets]}")
print(f"  - CX blocks CX count: {2*opt.cx_block.num_gates}")
display(opt.phase_block.to_svg(scale=0.4))
opt.cx_block.draw(figsize=(2, 2))

If we further conjugate the circuit by a CX gate with control 2 and target 5, the cost changes as follows:

- we remove 4 CX gates from the phase block (3rd and 5th phase gadget lose a leg);
- we add 2 CX gates to the phase block (4th phase gadget gains a leg);
- we add 2 CX gates to the CX blocks;

Overall, the cost is reduced by 0. It would be reduced by `2*circuit_rep-2` in the general case.

In [ ]:
opt.flip_cx(0, 2, 5)
cost = cost_fun(opt.phase_block, opt.cx_block)
print(f"Cost after second flip: {cost}")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.phase_block.gadgets]}")
print(f"  - CX blocks CX count: {2*opt.cx_block.num_gates}")
display(opt.phase_block.to_svg(scale=0.4))
opt.cx_block.draw(figsize=(2, 2))

## Annealing (to be completed)

The method `PhaseCircuitOptimizeranneal(num_iters, temp_schedule, cost_fun)` allows a given number of iterations of simulated annealing to be performed on the circuit, for a given temperature schedule and cost function.

The method also accepts an optional keyword argument `loggers`, for detailed logging.

In [ ]:
from math import log10, ceil
def log_iter(it, prev_cost, new_cost, accepted, flip, t, num_iters):
    if new_cost < prev_cost:
        print(f"Iter #{it:>0{ceil(log10(num_iters-1))}}, new cost: {new_cost}")
loggers = {
    "log_init_cost": lambda cost, num_iters: print(f"Init cost: {cost}"),
    "log_iter": log_iter,
    "log_final_cost": lambda cost, num_iters: print(f"Final cost: {cost}"),
}

...

In [ ]:
num_iters = 100
topology = Topology.grid(3, 3)
opt = PhaseCircuitOptimizer(orig_circuit, topology, 1, rng_seed=1)
temp_schedule = geometric_temp_schedule(t_init=1, t_final=1e-5)
cost_fun = cx_count_cost_fun(topology, circuit_rep=1)
opt.anneal(num_iters, temp_schedule, cost_fun, loggers=loggers)

...

In [ ]:
cost = cost_fun(opt.phase_block, opt.cx_block)
print(f"Cost after annealing: {cost}")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.phase_block.gadgets]}")
print(f"  - CX blocks CX count: {2*opt.cx_block.num_gates}")
display(opt.phase_block.to_svg(scale=0.8))
opt.cx_block.draw(figsize=(3, 3))

...

In [ ]:
num_iters = 1000
topology = Topology.grid(3, 3)
opt = PhaseCircuitOptimizer(orig_circuit, topology, 4, rng_seed=1)
temp_schedule = geometric_temp_schedule(t_init=2, t_final=1e-5)
cost_fun = cx_count_cost_fun(topology, circuit_rep=1)
opt.anneal(num_iters, temp_schedule, cost_fun, loggers=loggers)

...

In [ ]:
init_cost = cost_fun(orig_circuit, CXCircuit(topology))
print(f"Cost before annealing: {init_cost}")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.original_gadgets]}")
print()
final_cost = cost_fun(opt.phase_block, opt.cx_block)
print(f"Cost after annealing: {final_cost} ({(final_cost-init_cost)/init_cost:.0%})")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.phase_block.gadgets]}")
print(f"  - CX blocks CX count: {2*opt.cx_block.num_gates}")
display(opt.phase_block.to_svg(scale=0.8))
opt.cx_block.draw(figsize=(3, 3))

...

In [ ]:
num_iters = 1000
topology = Topology.grid(5, 5)
large_circuit = PhaseCircuit.random(topology.qubits, 50, min_legs=1, max_legs=3, rng_seed=0)
opt = PhaseCircuitOptimizer(large_circuit, topology, 5, rng_seed=1)
temp_schedule = straight_temp_schedule(t_init=1, t_final=1e-5)
cost_fun = cx_count_cost_fun(topology, circuit_rep=5)
opt.anneal(num_iters, temp_schedule, cost_fun, loggers=loggers)

...

In [ ]:
init_cost = cost_fun(large_circuit, CXCircuit(topology))
print(f"Cost before annealing: {init_cost}")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.original_gadgets]}")
print()
final_cost = cost_fun(opt.phase_block, opt.cx_block)
print(f"Cost after annealing: {final_cost} ({(final_cost-init_cost)/init_cost:.0%})")
print(f"  - Gadget CX counts: {[sum(cx_count(gadget, topology)[0]) for gadget in opt.phase_block.gadgets]}")
print(f"  - CX blocks CX count: {2*opt.cx_block.num_gates}")
opt.cx_block.draw(figsize=(3, 3))